# S3 에 저장된 모델을 SageMaker INF2 에 배포하기

이 노트북은 [MLP-KTLim/llama-3-Korean-Bllossom-8B](https://huggingface.co/MLP-KTLim/llama-3-Korean-Bllossom-8B) 의 한국어 파인 튜닝 모델을 AWS Neuron Model 로 바꾼 [Gonsoo/AWS-HF-optimum-neuron-0-0-28-llama-3-Korean-Bllossom-8B](https://huggingface.co/MLP-KTLim/llama-3-Korean-Bllossom-8B) 모델을 사용 합니다.<br>
AWS Hugging Face Text Generation Inference (HF TGI) 도커 컨테이너를 이용하여, SageMaker Endpoint 에 배포 합니다.

### 커널 설정
- 쥬피터 노트북의 오른쪽 상단의 conda_neuron_pytorch_p38 를 사용합니다.


#### 코드 참조
- [Deploy Llama 3 70B on AWS Inferentia2 with Hugging Face Optimum](https://github.com/philschmid/huggingface-inferentia2-samples/blob/main/llama3-70b/deploy-llama-3-70b-inferentia2.ipynb)

---

## 1. 다음과 같은 패키지를 설치 합니다.
- 처음 실행시에 아래의 주석을 제거하고 실행 하세요



In [1]:
# ! pip install huggingface_hub
# !pip install "sagemaker>=2.199.0" transformers --upgrade --quiet

## 2. SageMaker 세션 등 가져오기

In [2]:
import sagemaker
import boto3
sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)
region = sess.boto_region_name
print(f"sagemaker role arn: {role}")
print(f"sagemaker session region: {region}")


/home/ec2-user/anaconda3/envs/neuron_pytorch_p38/lib/python3.8/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[05/25/25 08:32:24] INFO     Found credentials from IAM Role:                                   ]8;id=115334;file:///home/ec2-user/anaconda3/envs/neuron_pytorch_p38/lib/python3.8/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=876823;file:///home/ec2-user/anaconda3/envs/neuron_pytorch_p38/lib/python3.8/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


                    INFO     Found credentials from IAM Role:                                   ]8;id=983598;file:///home/ec2-user/anaconda3/envs/neuron_pytorch_p38/lib/python3.8/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=765696;file:///home/ec2-user/anaconda3/envs/neuron_pytorch_p38/lib/python3.8/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

[05/25/25 08:32:25] INFO     Found credentials from IAM Role:                                   ]8;id=420454;file:///home/ec2-user/anaconda3/envs/neuron_pytorch_p38/lib/python3.8/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=408290;file:///home/ec2-user/anaconda3/envs/neuron_pytorch_p38/lib/python3.8/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

                    INFO     Found credentials from IAM Role:                                   ]8;id=891198;file:///home/ec2-user/anaconda3/envs/neuron_pytorch_p38/lib/python3.8/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=381361;file:///home/ec2-user/anaconda3/envs/neuron_pytorch_p38/lib/python3.8/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

sagemaker role arn: arn:aws:iam::057716757052:role/workshop-sagemaker-kfp-role2
sagemaker session region: us-west-2


## 3. Hugging Face LLM Inf2 DLC (Deep Learning Container ) 가져오기

새로운 Hugging Face TGI Neuronx DLC를 사용하여 AWS Inferentia2에서 추론을 실행할 수 있습니다. sagemaker SDK의 get_huggingface_llm_image_uri 메서드를 사용하여 원하는 backend, session, region, 그리고 version에 따라 적절한 Hugging Face TGI Neuronx DLC URI를 검색할 수 있습니다. 사용 가능한 모든 버전은 여기에서 확인할 수 있습니다.

**<u>"여기서는 2025.05.25" 현재 가장 최신인 SageMaker TGI 이미지를 사용합니다."</u>**

- 참조: [Hugging Face TGI Neuronx DLC URI](https://github.com/aws/deep-learning-containers/releases?q=tgi+AND+neuronx&expanded=true)



In [3]:
# TODO: Comment in when released
from sagemaker.huggingface import get_huggingface_llm_image_uri

# retrieve the llm image uri
# llm_image = get_huggingface_llm_image_uri(
#   "huggingface-neuronx",
#   ## version="0.0.22"
# )

llm_image = f"763104351884.dkr.ecr.{region}.amazonaws.com/huggingface-pytorch-tgi-inference:2.1.2-optimum0.0.28-neuronx-py310-ubuntu22.04-v1.2"
print(f"llm image uri: \n {llm_image}")

llm image uri: 
 763104351884.dkr.ecr.us-west-2.amazonaws.com/huggingface-pytorch-tgi-inference:2.1.2-optimum0.0.28-neuronx-py310-ubuntu22.04-v1.2


## 4. HF 에서 모델 다운로드 및 S3 업로드

### 모델 다운로드

In [4]:
import os

download_folder = "/home/ec2-user/SageMaker/models/AWS-HF-optimum-neuron-0-0-28-llama-3-Korean-Bllossom-8B"

# model_name = "Gonsoo/AWS-Neuron-llama-3-Korean-Bllossom-8B" # Neuron-CC 2.13 버전 에서 컴파일
# 아래의 도커의 optimum-cli 를 통하여 컴파일 됨
# 763104351884.dkr.ecr.us-east-1.amazonaws.com/huggingface-pytorch-tgi-inference:2.1.2-optimum0.0.24-neuronx-py310-ubuntu22.04-v1.0
model_name = "Gonsoo/AWS-HF-optimum-neuron-0-0-28-llama-3-Korean-Bllossom-8B" # Neuron-CC 2.14 버전 에서 컴파일

os.makedirs(download_folder, exist_ok=True)


In [5]:
! huggingface-cli download {model_name} --local-dir {download_folder} --exclude ".cache" --exclude ".gitattributes"
! ls -al {download_folder} 

Fetching 8 files: 100%|█████████████████████████| 8/8 [00:00<00:00, 3701.54it/s]
/home/ec2-user/SageMaker/models/AWS-HF-optimum-neuron-0-0-28-llama-3-Korean-Bllossom-8B
total 16892
drwxrwxr-x 4 ec2-user ec2-user     4096 May 25 03:30 .
drwxrwxr-x 4 ec2-user ec2-user     4096 May 25 03:30 ..
drwxrwxr-x 3 ec2-user ec2-user     4096 May 24 08:21 .cache
drwxrwxr-x 2 ec2-user ec2-user     4096 May 24 08:21 compiled
-rw-rw-r-- 1 ec2-user ec2-user     1112 May 24 08:21 config.json
-rw-rw-r-- 1 ec2-user ec2-user      172 May 24 08:21 generation_config.json
-rw-rw-r-- 1 ec2-user ec2-user     1490 May 25 03:30 README.md
-rw-rw-r-- 1 ec2-user ec2-user      444 May 24 08:21 special_tokens_map.json
-rw-rw-r-- 1 ec2-user ec2-user    51136 May 24 08:21 tokenizer_config.json
-rw-rw-r-- 1 ec2-user ec2-user 17209962 May 24 08:21 tokenizer.json


### 모델을 S3 에 업로드

In [6]:
desired_s3_uri = f"s3://{sagemaker_session_bucket}/inf2_model"

In [7]:
from sagemaker.s3 import S3Uploader

file_s3_path = S3Uploader.upload(local_path=download_folder, desired_s3_uri=desired_s3_uri)
print("file_s3_path: ", file_s3_path)


[05/25/25 08:32:26] INFO     Found credentials from IAM Role:                                   ]8;id=574770;file:///home/ec2-user/anaconda3/envs/neuron_pytorch_p38/lib/python3.8/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=509652;file:///home/ec2-user/anaconda3/envs/neuron_pytorch_p38/lib/python3.8/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

file_s3_path:  s3://sagemaker-us-west-2-057716757052/inf2_model


In [8]:
# ! aws s3 rm {file_s3_path} --recursive
! aws s3 ls {file_s3_path} --recursive

2025-05-25 08:32:28          1 inf2_model/.cache/huggingface/.gitignore
2025-05-25 08:32:28          0 inf2_model/.cache/huggingface/download/README.md.lock
2025-05-25 08:32:28        101 inf2_model/.cache/huggingface/download/README.md.metadata
2025-05-22 01:40:58          0 inf2_model/.cache/huggingface/download/compiled/2ae6fb8fd3c66e17e30f.neff.lock
2025-05-22 01:40:58        125 inf2_model/.cache/huggingface/download/compiled/2ae6fb8fd3c66e17e30f.neff.metadata
2025-05-25 08:32:28          0 inf2_model/.cache/huggingface/download/compiled/6262be51029c3eea47a4.neff.lock
2025-05-25 08:32:28        125 inf2_model/.cache/huggingface/download/compiled/6262be51029c3eea47a4.neff.metadata
2025-05-22 01:40:58          0 inf2_model/.cache/huggingface/download/compiled/675e2498ec72a928a923.neff.lock
2025-05-22 01:40:59        125 inf2_model/.cache/huggingface/download/compiled/675e2498ec72a928a923.neff.metadata
2025-05-25 08:32:28          0 inf2_model/.cache/huggingface/download/compiled/bc1

## 5. 한국어 모델을 inferentia2 에 배포하기


모델을 Amazon SageMaker에 배포하기 전에 TGI Neuronx 엔드포인트 구성을 정의해야 합니다. 다음과 같은 추가 매개변수를 정의해야 합니다:
- HF_MODEL_ID: "/opt/ml/model"
    - ECR 에서 추론 이미지를 다운로드 받고, 컨테이너가 생성이 됩니다. 이후에 S3 의 모델 파일은 "/opt/ml/model" 로 다운로드가 됩니다. 그래서 이 경로를 기술해야만 합니다.
- HF_NUM_CORES: 컴파일에 사용된 Neuron 코어의 수.
- HF_BATCH_SIZE: 모델 컴파일에 사용된 배치 크기.
- HF_SEQUENCE_LENGTH: 모델 컴파일에 사용된 시퀀스 길이.
- HF_AUTO_CAST_TYPE: 모델 컴파일에 사용된 자동 캐스트 유형.

또한 기존의 TGI 매개변수도 정의해야 합니다:

- HF_MODEL_ID: Hugging Face 모델 ID.
- MAX_BATCH_SIZE: 모델이 처리할 수 있는 최대 배치 크기로, 컴파일에 사용된 배치 크기와 동일합니다.
- MAX_INPUT_LENGTH: 모델이 처리할 수 있는 최대 입력 길이.
- MAX_TOTAL_TOKENS: 모델이 생성할 수 있는 최대 총 토큰 수로, 컴파일에 사용된 시퀀스 길이와 동일합니다.

### 모델 배포 파라미터 설정 및 SageMaker HuggingFaceModel 생성

In [9]:
from huggingface_hub import HfFolder
from sagemaker.huggingface import HuggingFaceModel

# sagemaker config
instance_type = "ml.inf2.xlarge"
health_check_timeout=600 # additional time to load the model
volume_size=64 # size in GB of the EBS volume

# Define Model and Endpoint configuration parameter
config = {
    "HF_MODEL_ID": "/opt/ml/model",       # Path to the model in the container
    "HF_NUM_CORES": "2", # number of neuron cores
    "HF_BATCH_SIZE": "4", # batch size used to compile the model
    "HF_SEQUENCE_LENGTH": "4096", # length used to compile the model
    "HF_AUTO_CAST_TYPE": "fp16",  # dtype of the model
    "MAX_BATCH_SIZE": "4", # max batch size for the model
    "MAX_INPUT_LENGTH": "2048", # max length of input text
    "MAX_TOTAL_TOKENS": "4096", # max length of generated text
    "MESSAGES_API_ENABLED": "true", # Enable the messages API
}

# model_data 딕셔너리 생성
model_data = {
    'S3DataSource': {
        'S3Uri': f"{file_s3_path}/",
        'S3DataType': 'S3Prefix',
        'CompressionType': 'None'
    }
}

print("model_data: \n", model_data) 
# create HuggingFaceModel with the image uri

llm_model = HuggingFaceModel(
  role=role,
  image_uri=llm_image,
  model_data=model_data,
  env=config
)

model_data: 
 {'S3DataSource': {'S3Uri': 's3://sagemaker-us-west-2-057716757052/inf2_model/', 'S3DataType': 'S3Prefix', 'CompressionType': 'None'}}


[05/25/25 08:32:28] INFO     Found credentials from IAM Role:                                   ]8;id=825333;file:///home/ec2-user/anaconda3/envs/neuron_pytorch_p38/lib/python3.8/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=499573;file:///home/ec2-user/anaconda3/envs/neuron_pytorch_p38/lib/python3.8/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

### 모델을 SageMaker Endpoint 에 배포

HuggingFaceModel을 생성한 후에는 deploy 메서드를 사용하여 Amazon SageMaker에 배포할 수 있습니다. 우리는 ml.inf2.xlarge 인스턴스 유형을 사용하여 모델을 배포할 것입니다. TGI는 자동으로 모든 Inferentia 디바이스에 걸쳐 모델을 분산하고 샤딩할 것입니다.
- 다음은 약 17분이 소요 됩니다.

In [ ]:
%%time

# deactivate warning since model is compiled
llm_model._is_compiled_model = True

from datetime import datetime

timestamp = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
# endpoint_name 생성 (sm-llama3-kr-inf2-yyyy-mm-dd-hh-mm-ss 형식)
endpoint_name = f"sm-llama3-kr-inf2-{timestamp}"


llm = llm_model.deploy(
  initial_instance_count=1,
  instance_type=instance_type,
  container_startup_health_check_timeout=health_check_timeout,
  volume_size=volume_size,
  endpoint_name=endpoint_name
)

[05/25/25 08:32:29] INFO     Creating model with name:                                              ]8;id=887891;file:///home/ec2-user/anaconda3/envs/neuron_pytorch_p38/lib/python3.8/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=299991;file:///home/ec2-user/anaconda3/envs/neuron_pytorch_p38/lib/python3.8/site-packages/sagemaker/session.py#4094\4094]8;;\
                             huggingface-pytorch-tgi-inference-ml-in-2025-05-25-08-32-29-130                       

                    INFO     Creating endpoint-config with name                                     ]8;id=344230;file:///home/ec2-user/anaconda3/envs/neuron_pytorch_p38/lib/python3.8/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=133003;file:///home/ec2-user/anaconda3/envs/neuron_pytorch_p38/lib/python3.8/site-packages/sagemaker/session.py#6019\6019]8;;\
                             sm-llama3-kr-inf2-2025-05-25-08-32-28                                                 

[05/25/25 08:32:30] INFO     Creating endpoint with name sm-llama3-kr-inf2-2025-05-25-08-32-28      ]8;id=160153;file:///home/ec2-user/anaconda3/envs/neuron_pytorch_p38/lib/python3.8/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=192573;file:///home/ec2-user/anaconda3/envs/neuron_pytorch_p38/lib/python3.8/site-packages/sagemaker/session.py#4841\4841]8;;\

----------------------------!CPU times: user 424 ms, sys: 42.7 ms, total: 467 ms
Wall time: 15min 4s


## 6. 추론

엔드포인트가 배포된 후에는 이를 통해 추론을 실행할 수 있습니다. 우리는 predictor의 predict 메서드를 사용하여 엔드포인트에서 추론을 실행할 것입니다. 생성에 영향을 미치는 다양한 매개변수로 추론을 수행할 수 있습니다. 매개변수는 페이로드의 parameters 속성에서 정의할 수 있습니다. 지원되는 매개변수는 여기에서 확인할 수 있습니다.
메시지 API를 사용하면 대화 방식으로 모델과 상호작용할 수 있습니다. 메시지의 역할과 내용을 정의할 수 있습니다. 역할은 system, assistant 또는 user일 수 있습니다. system 역할은 모델에 컨텍스트를 제공하는 데 사용되고, user 역할은 질문을 하거나 모델에 입력을 제공하는 데 사용됩니다.

```json
{
  "messages": [
    { "role": "system", "content": "You are a helpful assistant." },
    { "role": "user", "content": "What is deep learning?" }
  ]
}
```

## 6.1. Message API 로 추론

In [ ]:
# Prompt to generate
messages=[
    { "role": "system", "content": "당신은 인공지능 전문가 입니다." },
    { "role": "user", "content": "딥러닝이 무엇인지 말해 주세요?" }
  ]

# Generation arguments
parameters = {
    "model": "Gonsoo/AWS-Neuron-llama-3-Korean-Bllossom-8B", # placholder, needed
    "top_p": 0.6,
    "temperature": 0.9,
    "max_tokens": 2048,
    "stop": ["<|eot_id|>"],
}

이제 테스트 해보시죠

In [ ]:
%%time
chat = llm.predict({"messages" :messages, **parameters,"steam":True})

print(chat["choices"][0]["message"]["content"].strip())

딥러닝(Deep Learning)은 인공지능(AI)의 한 분야로, 인공 신경망(Artificial Neural Networks)을 이용하여 데이터를 학습시키는 방법입니다. 딥러닝은 인간의 뇌를 모방하여 데이터를 처리하고 학습하는 방식으로, 여러 층의 신경망을 통해 복잡한 패턴을 인식하고 학습할 수 있습니다.

딥러닝은 다음과 같은 특징을 가지고 있습니다:

1. **다층 신경망**: 딥러닝은 여러 층의 신경망을 사용하여 데이터를 처리합니다. 각 층은 입력 데이터를 더 높은 수준의 추상화로 변환합니다.
2. **자동 추론**: 딥러닝은 데이터로부터 학습하여 모델을 자동으로 최적화합니다. 사람이 직접 규칙을 정의하지 않아도 모델이 데이터를 통해 학습할 수 있습니다.
3. **대규모 데이터 필요**: 딥러닝 모델은 대규모의 데이터가 필요합니다. 많은 데이터를 통해 모델이 더 정확하게 학습할 수 있습니다.
4. **컴퓨터 비전, 음성 인식, 자연어 처리 등 다양한 응용**: 딥러닝은 컴퓨터 비전, 음성 인식, 자연어 처리 등 다양한 분야에서 응용될 수 있습니다.

딥러닝은 다음과 같은 기술로 구성됩니다:

1. **Convolutional Neural Networks (CNN)**: 이미지 처리에 주로 사용됩니다.
2. **Recurrent Neural Networks (RNN)**: 시계열 데이터나 언어 처리에 주로 사용됩니다.
3. **Generative Adversarial Networks (GAN)**: 새로운 데이터를 생성하는 데 사용됩니다.

딥러닝은 의료 진단, 자율 주행, 추천 시스템 등 다양한 분야에서 혁신적인 성과를 내고 있으며, 앞으로도 지속적으로 발전할 것으로 기대됩니다.<|eot_id|>
CPU times: user 10.6 ms, sys: 1.14 ms, total: 11.8 ms
Wall time: 18.1 s


## 6.2. Streaming 추론

In [ ]:
import boto3
import json

runtime = boto3.client('sagemaker-runtime')

payload = {
    "messages": messages,
    **parameters,
    "stream": True
}

response = runtime.invoke_endpoint_with_response_stream(
    EndpointName=endpoint_name,
    ContentType='application/json',
    Body=json.dumps(payload)
)

# 수정된 스트리밍 응답 처리
for event in response['Body']:
    try:
        # Bytes를 string으로 decode
        chunk_bytes = event['PayloadPart']['Bytes']
        chunk_str = chunk_bytes.decode('utf-8')
        
        # 빈 문자열이나 공백만 있는 경우 건너뛰기
        if not chunk_str.strip():
            continue
            
        # SSE 형태 데이터 처리 (data: 접두사 제거)
        if chunk_str.startswith('data: '):
            chunk_str = chunk_str[6:]
            
        # [DONE] 메시지는 스트림 종료 신호
        if chunk_str.strip() == '[DONE]':
            break
            
        # JSON 파싱
        chunk = json.loads(chunk_str)
        
        # OpenAI 호환 형태 처리
        if 'choices' in chunk:
            delta = chunk['choices'][0].get('delta', {})
            if 'content' in delta:
                print(delta['content'], end='', flush=True)
        
        # TGI 원본 형태 처리 (token 기반)
        elif 'token' in chunk:
            if not chunk.get('finished', False):
                print(chunk['token']['text'], end='', flush=True)
                
    except json.JSONDecodeError as e:
        # JSON 파싱 실패 시 건너뛰기 (또는 디버깅용 출력)
        # print(f"JSON 파싱 에러: {chunk_str[:100]}")  # 디버깅용
        continue
    except KeyError:
        # 예상하지 못한 키 구조는 건너뛰기
        continue
    except Exception as e:
        # 기타 예외는 건너뛰기
        # print(f"기타 에러: {e}")  # 디버깅용
        continue

print()  # 마지막에 줄바꿈

[05/25/25 08:47:51] INFO     Found credentials from IAM Role:                                   ]8;id=70131;file:///home/ec2-user/anaconda3/envs/neuron_pytorch_p38/lib/python3.8/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=410084;file:///home/ec2-user/anaconda3/envs/neuron_pytorch_p38/lib/python3.8/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

딥러닝(Deep Learning)은 인공지능(AI)의 한 분야로, 인공 신경망(Artificial Neural Network)을 사용하여 데이터를 학습하고 예측하거나 분류하는 기술입니다. 딥러닝은 인공 신경망의 여러 층을 쌓아 올려, 복잡한 패턴을 학습할 수 있도록 설계되었습니다. 이러한 층을 히든 레이어(Hidden Layers)라고 하며, 이 레이어들이 데이터를 여러 차례 처리하여 더 깊은 의미를 추출.

딥러닝은 주로 다음과 같은 특징을 있습니다:

1. ** 학습**: 딥러닝 모델은 대량의 데이터를 학습하여 패턴을 학습합니다. 이 과정에서 모델은 데이터의 특징을 자동으로 추출하고, 이를 통해 예측이나 분류를 수행합니다.
2. **자동 특징 추출**: 인간이 수작업으로 특징을 추출하지 않아도, 딥러닝 모델은 데이터 자체에서 중요한 특징을 자동으로 추출합니다.
3. **복잡한 문제 해결**: 딥러닝은 이미지 인식, 음성 인식, 자연어 처리 등 복잡한 문제를 해결하는 데 효과적입니다.
4. **다양한 응용 분야**: 딥러닝은 컴퓨터 비전, 음성 인식, 자연어 처리, 게임, 자율 주행 등 다양한 분야에서 활용됩니다.

딥러닝의 대표적인 알고리즘으로는 다음과 같은 것들이 있습니다:

1. **Convolutional Neural Networks (CNN)**: 이미지 처리에 특화된 딥러닝 알고리즘으로, 주로 컴퓨터 비전 분야에서 사용됩니다.
2. **Recurrent Neural Networks (RNN)**: 시계열 데이터나 언어 처리에 적합한 딥러닝 알고리즘으로, 주로 자연어 처리 분야에서 사용됩니다.
3. **Generative Adversarial Networks (GAN)**: 생성 모델로, 새로운 데이터를 생성하거나 데이터의 품질을 향상시키는 데 사용됩니다.

딥러닝은 인공지능의 한 분야이지만, 최근에는 AI의 부분을 차지하고 있으며, 다양한 산업 분야에서 혁신적인 변화를 이끌고 있습니다.


## 6. 리소스 정리

SageMaker HuggingFace Model 및 엔드포인트를 삭제 합니다.


In [ ]:
llm.delete_endpoint()

[05/25/25 08:48:12] INFO     Deleting endpoint configuration with name:                             ]8;id=62075;file:///home/ec2-user/anaconda3/envs/neuron_pytorch_p38/lib/python3.8/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=365469;file:///home/ec2-user/anaconda3/envs/neuron_pytorch_p38/lib/python3.8/site-packages/sagemaker/session.py#4995\4995]8;;\
                             sm-llama3-kr-inf2-2025-05-25-08-32-28                                                 

                    INFO     Deleting endpoint with name: sm-llama3-kr-inf2-2025-05-25-08-32-28     ]8;id=411233;file:///home/ec2-user/anaconda3/envs/neuron_pytorch_p38/lib/python3.8/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=299855;file:///home/ec2-user/anaconda3/envs/neuron_pytorch_p38/lib/python3.8/site-packages/sagemaker/session.py#4985\4985]8;;\